In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
test=pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
sample_submission.head()

In [ ]:
train.info()

In [ ]:
train.text=train.text.apply(lambda x:x.lower() )
test.text=test.text.apply(lambda x:x.lower())

In [ ]:
train.text[0]

In [ ]:
import re

In [ ]:
#removing any urls
train.text=train.text.apply(lambda x:re.sub('^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', x) )
test.text=test.text.apply(lambda x:re.sub('^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', '', x) )

In [ ]:
#remove email ids
train.text=train.text.apply(lambda x:re.sub('^(.+)@(\\S+)$', '', x) )
test.text=test.text.apply(lambda x:re.sub('^(.+)@(\\S+)$', '', x) )

In [ ]:
#remove brackets
train.text=train.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
test.text=test.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
train.text=train.text.apply(lambda x:re.sub('<.*?>+', '', x) )
test.text=test.text.apply(lambda x:re.sub('<.*?>+', '', x) )

In [ ]:
#remove space
train.text=train.text.apply(lambda x:re.sub('\n', '', x) )
test.text=test.text.apply(lambda x:re.sub('\n', '', x) )

In [ ]:
#remove punctuations
import string
train.text=train.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
test.text=test.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )

In [ ]:
train.text[10]

In [ ]:
train.text

In [ ]:
#words with numbers to be removed
train.text=train.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\w*\d\w*', '', x) )


In [ ]:
train.text

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:

    print(train[train.target==1].keyword.sample(50) )

In [ ]:
for i in range(7613):
    if train.iloc[i].target==1 and type(train.iloc[i].keyword)==float:
        train.at[[i],['keyword']]='casualty'
        
        

In [ ]:
train.head()

In [ ]:
    print(train[train.target==0].keyword.sample(20) )

In [ ]:
for i in range(7613):
    if train.iloc[i].target==0 and type(train.iloc[i].location)==float:
        train.at[[i],['keyword']]='refugees'
        

In [ ]:
    print(train[train.target==0].keyword )

In [ ]:
train['location'].fillna('USA',inplace=True)

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
#GIVES THE TOKEN BASED ON REGEXP
token=nltk.tokenize.RegexpTokenizer(r'\w+')
train.text=train.text.apply(lambda x:token.tokenize(x))
test.text=test.text.apply(lambda x:token.tokenize(x))

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
#removing stop words
nltk.download('stopwords')
train.text=train.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])
test.text=test.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])
train.text.head()

In [ ]:
stemmer = nltk.stem.PorterStemmer()
train.text=train.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))
test.text=test.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))
train.text.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer

In [ ]:
#we require to convert the words into nos so we use countvector

count_vectorizer = CountVectorizer()
train_vectors_count = count_vectorizer.fit_transform(train['text'])
test_vectors_count = count_vectorizer.transform(test["text"])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_vectors_count)
test_tfidf = tfidf_transformer.fit_transform(test_vectors_count)



In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(X_train_tfidf,train.target,test_size=0.3,random_state=42)

In [ ]:
train_vectors_count

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors=7)
clf = knn.fit(x_tr,y_tr)
predicted = knn.predict(x_val)
predicted

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(predicted,y_val))
print(confusion_matrix(predicted,y_val))

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf') 
clf.fit(x_tr, y_tr)
y_pred = clf.predict(x_val)

In [ ]:
print(classification_report(y_pred,y_val))
print(confusion_matrix(y_pred,y_val))

In [ ]:

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_tr, y_tr)
y_pred_class = nb.predict(x_val)

In [ ]:
print(classification_report(y_pred_class,y_val))
print(confusion_matrix(y_pred_class,y_val))

In [ ]:
#final predictions using SVM
final_predictions=clf.predict(test_tfidf)
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample_submission['target']=final_predictions

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission1.csv', index=False)